# Token Classification

token classification 문장 내에 있는 각 토큰마다 label을 부여하여 분류하는 task를 가리킵니다.대표적으로 NER(Named Entity Recognition)이 존재합니다.

NER task는 text에 있는 이름이 붙여진(named) entity를 인식하는 task입니다.
예를 들어 text내에서 사람의 이름, 건물의 이름, 도시의 이름, 위치, 기관 등을 인식하고 분류하는 task라고 보면됩니다.

[예시](ttps://huggingface.co/datasets/klue/viewer/ner/train)를 보겠습니다.

```
KLUE-ner dataset

특히 <영동고속도로:LC> <강릉:LC> 방향 <문막휴게소:LC>에서 <만종분기점:LC>까지 <5㎞:QT> 구간에는 승용차 전용 임시 갓길차로제를 운영하기로 했다.
```

영동고속도로, 강릉, 문막휴게소, 만종분기점은 장소(위치)를 가리킵니다. 따라서 LC라는 label이 붙어있습니다.

5km는 숫자 단위값입니다. QT라는 label이 붙어있습니다.

named되지 않은 entity에 대해서는 아무것도 없다는 것을 뜻하는 O라는 label이 붙습니다.

본격적으로 dataset을 활용해 NER task를 수행해보겠습니다.

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.3 MB/s 
     |████████████████████████████████| 120 kB 63.3 MB/s 
     |████████████████████████████████| 6.6 MB 41.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 62.3 MB/s 
     |████████████████████████████████| 115 kB 67.6 MB/s 
     |████████████████████████████████| 127 kB 69.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


[KLUE-NER](https://huggingface.co/datasets/klue) 데이터셋을 사용합니다.
* [KLUE](https://klue-benchmark.com/)는 GLUE와 같은 benchmark 데이터셋입니다.
* 한국어 model의 성능 평가를 위해 만들어진 범용적 데이터셋입니다.



In [ ]:
from datasets import load_dataset

dataset = load_dataset('klue', 'ner')

Generating train split:   0%|          | 0/21008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/ner/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 21008
    })
    validation: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 5000
    })
})


In [ ]:
train_data = dataset['train']
test_data = dataset['validation']

In [ ]:
for sent, token, ner_tag in zip(train_data['sentence'], train_data['tokens'], train_data['ner_tags']):
    print(f'sentence: {sent}')
    print(f'tokens: {token}')
    print(f'ner_tags: {ner_tag}')

    break

sentence: 특히 <영동고속도로:LC> <강릉:LC> 방향 <문막휴게소:LC>에서 <만종분기점:LC>까지 <5㎞:QT> 구간에는 승용차 전용 임시 갓길차로제를 운영하기로 했다.
tokens: ['특', '히', ' ', '영', '동', '고', '속', '도', '로', ' ', '강', '릉', ' ', '방', '향', ' ', '문', '막', '휴', '게', '소', '에', '서', ' ', '만', '종', '분', '기', '점', '까', '지', ' ', '5', '㎞', ' ', '구', '간', '에', '는', ' ', '승', '용', '차', ' ', '전', '용', ' ', '임', '시', ' ', '갓', '길', '차', '로', '제', '를', ' ', '운', '영', '하', '기', '로', ' ', '했', '다', '.']
ner_tags: [12, 12, 12, 2, 3, 3, 3, 3, 3, 12, 2, 3, 12, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]


각 tag와 mapping되는 id는 다음과 같습니다.

* DT - 0,1
* LC - 2,3
* OG - 4,5
* PS - 6,7
* QT - 8,9
* TI - 10, 11
* O - 12

각 tag당 2개의 id가 mapping이 되는 이유는 각 tag의 시작을 알리기 위해서입니다. 
* 예를 들어, DT의 0은 DT tag의 시작을 뜻합니다. → 시작을 뜻하기 때문에 tag 앞에 'B-'를 붙입니다. e.g) B-DT
    * 5개월간 → [0, 1, 1, 1]

NER 데이터셋의 특이한 점은 이미 tokenization이 되었다는 것입니다. 하지만 model을 이용해 학습하기 위해서는 별도의 전처리가 필요합니다.
* 문장에 special token 추가하기
* token → token id로 바꾸는 것

활용할 model이 [distilbert-base-multilingual-cased](https://huggingface.co/distilbert-base-multilingual-cased)이기 때문에 distilbert에서 요구하는 입력 형태에 맞춰 데이터셋의 tokens를 전처리합니다.
* 문장의 시작: [CLS], 문장의 끝:[SEP]
* 각 token을 id값으로 바꾸기


model과 tokenizer를 load합니다.

In [ ]:
MODEL_NAME = 'distilbert-base-multilingual-cased'

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=13)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You s

tokenizer가 올바르게 작동하는 지 확인합니다.

In [ ]:
print(tokenizer(train_data['sentence'][0]))

{'input_ids': [101, 39671, 133, 9574, 18778, 11664, 43962, 54448, 131, 43586, 135, 133, 8853, 118904, 131, 43586, 135, 9328, 79544, 133, 9297, 118907, 119458, 14153, 22333, 131, 43586, 135, 24178, 133, 9248, 22200, 37712, 12310, 34907, 131, 43586, 135, 8939, 12508, 133, 100, 131, 154, 11090, 135, 72949, 15303, 9484, 24974, 23466, 9665, 24974, 9644, 14040, 8851, 118666, 23466, 11261, 53726, 91988, 22440, 11261, 23622, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Preprocess

전처리 함수를 만들어 model이 학습할 수 있는 입력 형태로 바꾸겠습니다.

In [ ]:
# 전처리 속도를 높이기 위해 [CLS], [SEP], [PAD] token id를 변수에 저장한다. → tokenizer.vocab을 이용하면 시간이 매우 오래 걸린다.

cls_token_id = tokenizer.vocab['[CLS]']
sep_token_id = tokenizer.vocab['[SEP]']
pad_token_id = tokenizer.vocab['[PAD]']

In [ ]:
def preprocess(tokens, ner_tag, max_length):
    length = len(tokens[:max_length-2])
    
    input_ids = [0] * length
    attention_mask = [1] * (length+2)
    # token_type_ids = [0] * max_length # unused in distilbert, used in bert

    tokens = tokens[:max_length-2]
    labels = [12] + ner_tag[:length] + [12]

    pre_word = '_'
    for i, token in enumerate(tokens):
        if token == ' ':
            pre_word = '_'
            token = '_'
        if pre_word != '_':
            token = '##' + token

        input_ids[i] = tokenizer.convert_tokens_to_ids(token)
        pre_word = token

    input_ids = [cls_token_id] + input_ids + [sep_token_id] 
    pad_length = max_length - len(input_ids)

    input_ids += [pad_token_id] * pad_length
    attention_mask += [0] * pad_length
    labels += [12] * pad_length

    # return {'input_ids':input_ids, 'token_type_ids':token_type_ids, 'attention_mask':attention_mask, 'labels':labels} # for bert
    return {'input_ids':input_ids, 'attention_mask':attention_mask, 'labels':labels} # distilbert

함수가 정상적으로 동작하는지 간단하게 살펴보겠습니다.
* km와 같은 단위를 제외하고는 대부분의 데이터셋의 tokens가 정상적으로 변환된 것을 확인할 수 있습니다.
* vocab에 존재하지않는 단어는 vocab에 token을 새로 추가할 수도 있습니다. 기존 vocab에 token을 새롭게 추가하는 것을 나중에 배워보도록 하겠습니다. → 새로운 tokenizer를 학습시켜 만드는 것도 방법입니다.

In [ ]:
inputs = preprocess(train_data['tokens'][0], train_data['ner_tags'][0], 80)

In [ ]:
for t_id, label in zip(inputs['input_ids'], inputs['labels']):
    print(tokenizer.convert_ids_to_tokens(t_id), label)

[CLS] 12
특 12
##히 12
_ 12
영 2
##동 3
##고 3
##속 3
##도 3
##로 3
_ 12
강 2
##릉 3
_ 12
방 12
##향 12
_ 12
문 2
##막 3
##휴 3
##게 3
##소 3
##에 12
##서 12
_ 12
만 2
##종 3
##분 3
##기 3
##점 3
##까 12
##지 12
_ 12
5 8
[UNK] 9
_ 12
구 12
##간 12
##에 12
##는 12
_ 12
승 12
##용 12
##차 12
_ 12
전 12
##용 12
_ 12
임 12
##시 12
_ 12
갓 12
##길 12
##차 12
##로 12
##제 12
##를 12
_ 12
운 12
##영 12
##하 12
##기 12
##로 12
_ 12
했 12
##다 12
##. 12
[SEP] 12
[PAD] 12
[PAD] 12
[PAD] 12
[PAD] 12
[PAD] 12
[PAD] 12
[PAD] 12
[PAD] 12
[PAD] 12
[PAD] 12
[PAD] 12
[PAD] 12


전처리 함수를 이용하여 데이터셋을 전처리하겠습니다.

In [ ]:
train_max_length = max([len(tokens) for tokens in train_data['tokens']]) + 2
test_max_length = max([len(tokens) for tokens in test_data['tokens']]) + 2

# 전처리를 위해 최대 길이를 미리 구한다.
print(train_max_length, test_max_length)

147 147


In [ ]:
from tqdm.notebook import tqdm

train_tokens = train_data['tokens']
train_labels = train_data['ner_tags']
test_tokens = test_data['tokens']
test_labels = test_data['ner_tags']

train_encodings = [preprocess(tokens, labels, train_max_length) for tokens, labels in tqdm(zip(train_tokens, train_labels), total=len(train_labels))]
test_encodings = [preprocess(tokens, labels, train_max_length) for tokens, labels in tqdm(zip(test_tokens, test_labels), total=len(test_labels))]

  0%|          | 0/21008 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

전처리된 데이터셋을 이용해 PyTorch Dataset을 만들도록 하겠습니다.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class NERDataset(Dataset):
    def __init__(self, encoding):
        self.encoding = encoding

    def __getitem__(self, idx):
        data = {k:torch.tensor(v) for k,v in self.encoding[idx].items()}

        return data

    def __len__(self):
        return len(self.encoding)

In [ ]:
train_dataset = NERDataset(train_encodings)
test_dataset = NERDataset(test_encodings)

Dataset이 정확하게 만들어졌는지 테스트 합니다.

In [ ]:
print(train_dataset[0])

{'input_ids': tensor([   101,   9891,  18108,    168,   9574,  18778,  11664,  43962,  12092,
          11261,    168,   8853, 118904,    168,   9328,  79544,    168,   9297,
         118907, 119458,  14153,  22333,  10530,  12424,    168,   9248,  22200,
          37712,  12310,  34907, 118671,  12508,    168,    126,    100,    168,
           8908,  18784,  10530,  11018,    168,   9484,  24974,  23466,    168,
           9665,  24974,    168,   9644,  14040,    168,   8851, 118666,  23466,
          11261,  17730,  11513,    168,   9606,  30858,  35506,  12310,  11261,
            168,   9965,  11903, 110864,    102,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0

## Train, Evaluate

Trainer, TrainingArguments 클래스를 이용해서 model을 학습시키고 평가할 것입니다.

먼저 metric을 구하는 함수를 만들겠습니다.

In [ ]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids.flatten() # metric이 1D array-like만 입력으로 받기 때문에 flatten한다.
    preds = pred.predictions.argmax(-1).flatten() # metric이 1D array-like만 입력으로 받기 때문에 flatten한다.

    f1 = f1_score(labels, preds, average='macro') 
    acc = accuracy_score(labels, preds)

    return {'accuracy':acc, 'f1':f1}

TrainingArguments를 정의합니다.

In [ ]:
training_args = TrainingArguments(
    output_dir = './outputs',
    logging_dir = './logs',
    num_train_epochs = 5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=3e-5,
    logging_steps = 100,
    save_steps=500,
    save_total_limit=2
)

Trainer를 정의합니다.

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [ ]:
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

model을 학습시킵니다.

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 21008
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3285


Step,Training Loss
100,0.278900
200,0.093400
300,0.073700
400,0.063700
500,0.055800
600,0.054400
700,0.049600
800,0.043400
900,0.043800
1000,0.039700


Saving model checkpoint to ./outputs/checkpoint-500
Configuration saved in ./outputs/checkpoint-500/config.json
Model weights saved in ./outputs/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-1000
Configuration saved in ./outputs/checkpoint-1000/config.json
Model weights saved in ./outputs/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-1500
Configuration saved in ./outputs/checkpoint-1500/config.json
Model weights saved in ./outputs/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [outputs/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./outputs/checkpoint-2000
Configuration saved in ./outputs/checkpoint-2000/config.json
Model weights saved in ./outputs/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [outputs/checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to ./outputs/checkpoint-2500
Configuration saved in ./outputs/checkpoint-2500/config.json
Mode

TrainOutput(global_step=3285, training_loss=0.042747495744144895, metrics={'train_runtime': 1592.1644, 'train_samples_per_second': 65.973, 'train_steps_per_second': 2.063, 'total_flos': 3941015794279200.0, 'train_loss': 0.042747495744144895, 'epoch': 5.0})

model을 평가합니다.

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5000
  Batch size = 32


(735000,)
(735000,)


{'eval_loss': 0.04650218412280083,
 'eval_accuracy': 0.9862190476190477,
 'eval_f1': 0.891516853009466,
 'eval_runtime': 23.0499,
 'eval_samples_per_second': 216.92,
 'eval_steps_per_second': 6.811}

## Inference

간단한 테스트 문장을 이용해 Inference를 수행합니다.

1. 먼저 tag_id와 tag가 어떻게 mapping이 되는지 정의합니다.
2. text를 tokenizer를 사용해 모델 입력에 맞게 변환한 뒤, 각 tensor를 2차원 형태로 변형합니다. → (batch_size, seq_len) 형태
    * distilbert가 2차원 형태의 입력 값을 받기 때문입니다.
3. model이 반환한 예측값을 tag 정보로 바꿉니다.
4. 각 token마다 어떤 tag가 붙는 지 출력합니다.

In [ ]:
# 'B-': 각 tag의 시작을 가리킵니다. 
tag_mapped = {0:'B-DT', 1:'DT',2:'B-LC',3:'LC',4:'B-OG',5:'OG',6:'B-PS',7:'PS',8:'B-QT',9:'QT',10:'B-TI',11:'TI',12:'O'}


def predict(text):
    tokenized = tokenizer(text)
    tokenized = {k:torch.tensor(v).to(device).unsqueeze(0) for k,v in tokenized.items()}

    with torch.no_grad():
        output = model(**tokenized)

    logits = output.logits.detach().cpu().numpy().squeeze().argmax(-1)
    logits = [tag_mapped[t_id] for t_id in logits]

    for token, tag in zip(tokenizer.tokenize(text), logits[1:-1]):
        print(token, tag)
    

In [ ]:
predict("2022년 08월 27일, TVN에서 10시에 첫방송합니다.")

2022 B-DT
##년 DT
08 DT
##월 DT
27일 DT
, O
TV B-OG
##N OG
##에서 O
10 B-TI
##시 TI
##에 O
첫 O
##방송 O
##합 O
##니다 O
. O
